In [ ]:
import numpy as np
import random
import math

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import rc
from IPython.display import HTML

## DLA Functions

In [ ]:
def stick(position, probability, grid, size):
    """ Check if point should stick or not """
    directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]
    stick = False
    for d in directions:
        surrounding = [position[0] + d[0], position[1] + d[1]]
        if surrounding[0] >= 0 and surrounding[0] < size and surrounding[1] >= 0 and surrounding[1] < size:
            p_stick = random.random()
            if grid[surrounding[0]][surrounding[1]] == 1 and p_stick <= probability:
                stick = True
                break
    return stick

def generate_grid(grid_size):
    """ Generate grid to check surroundings """
    g = [[0 for i in range(grid_size)] for j in range(grid_size)]
    g[int(grid_size / 2)][int(grid_size / 2)] = 1
    return g

def walk(position, step):
    """ Move point """
    return [position[0] + step[0], position[1] + step[1]]

def select_direction(position, size):
    """ Select possible directions for point to move """
    possible_directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]
    if position[0] == size:
        possible_directions.remove([1, 0])
    if position[1] == size:
        possible_directions.remove([0, 1])
    if position[0] == 0:
        possible_directions.remove([-1, 0])
    if position[1] == 0:
        possible_directions.remove([0, -1])
    direction = possible_directions[random.randint(0, len(possible_directions) - 1)]
    return direction

## Main DLA method

In [ ]:
def dla_run(size, n_points, probability):
    """ DLA run with given size, number of points and sticking probability"""
    print('Starting DLA run with S: %i | N: %i | P: %.2f' % (size, n_points, probability))
    grid = generate_grid(size)
    starting_positions = [[0, i] for i in range(size)] + [[i, 0] for i in range(size)]
    starting_positions += [[size, i] for i in range(size)] + [[i, size] for i in range(size)]
    points = [[int(size / 2), int(size / 2)]]

    for run in range(n_points):
        start = starting_positions[random.randint(0, len(starting_positions) - 1)]
        position = start
        walking = True

        while walking:
            step = select_direction(position, size)
            position = walk(position, step)
            if stick(position, probability, grid, size):
                walking = False
                points.append(position)
                grid[position[0]][position[1]] = 1
    points_x = [i[0] for i in points]
    points_y = [i[1] for i in points]
    print('Done!')
    return points_x, points_y

## Run DLA

In [ ]:
size = 50
n_points = 200
p = 1

x, y = dla_run(size, n_points, p)

## Initialize animation

In [ ]:
# Create new Figure and an Axes which fills it.
fig = plt.figure(figsize=(6, 6))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax.set_xlim(0, size)
ax.set_ylim(0, size)

In [ ]:
center = [size / 2, size / 2]
max_dist = math.sqrt((size) ** 2 + (size) ** 2)
max_dist = math.sqrt((size - center[0]) ** 2 + (size - center[1]) ** 2)
def norm_dist(new_point, center, max_dist):
    c_x, c_y = center
    x, y = new_point
    return math.sqrt((c_x - x) ** 2 + (c_y - y) ** 2) / max_dist

# Read positions
points = dict(position=[], size=50, color=[], lw=0.2, edgecolor='b', marker='o')
points['position'] = []
for i, j in zip(x, y):
    points['position'].append([i, j])

# Get colormap
cmap = matplotlib.cm.get_cmap('Spectral_r')

mid = [int(size / 2), int(size / 2)]
mid_color = cmap(norm_dist(mid, center, max_dist))
scat = ax.scatter(int(size / 2), int(size / 2), marker='o', s=points['size'], facecolors=mid_color)
point_count = 0

## Animate

In [ ]:
%matplotlib inline
interval = int(len(points['position']))

def update(frame_number):
    speed = int(n_points / 100)
    for i in range(speed):
        new_point = points['position'][frame_number * speed + i]
        dist = norm_dist(new_point, center, max_dist)
        color = cmap(dist)
        ax.scatter(new_point[0], new_point[1], marker='o', s=points['size'], facecolors=color)
        ax.set_title('Number of points: %i' % int((frame_number + 1) * speed))

# Construct the animation, using the update function as the animation director.
animation = FuncAnimation(fig, update, interval=interval)
# animation.save('dla.avi', dpi=200)
HTML(animation.to_html5_video())
rc('animation', html='html5')
animation